This is a program which is in testing phase. We will use a linear regression ML model to give an SOC number for the flights.

In [1]:
# Import the querying module
from flight_querying import query_flights
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Set up and retrieve the data from the database.
db_connect = query_flights()

In [2]:
test_id = 4620

# Call the data and get rid of the NA values.
df = db_connect.connect_flight_for_ml_data_prescription(test_id)
df = df[df["activity"] != "NA"].reset_index()

# # Add a dummy data to the end of the dataframe
# dummy_frame = pd.DataFrame({"time": 100, "id": 23232, "activity": "Ending", "soc": 0, "power" : 0})
# df.loc[len(df)] = dummy_frame

len(df)

38528

In [3]:
df.head()

,index,time,id,activity,soc,power
0,16096,13.500106,4620,takeoff,99,65
1,16097,13.500106,4620,takeoff,99,65
2,16098,13.501768,4620,takeoff,99,65
3,16099,13.501768,4620,takeoff,99,65
4,16100,13.503435,4620,takeoff,99,66


In [4]:
df.tail(1)

,index,time,id,activity,soc,power
38527,57299,47.998861,4620,landing,33,0


In [5]:
# Make the data dictionary holder
data_dict = {
    "Time Delta": [],
    "SOC Delta": [],
    "Activity": [],
    "Average Power": [],
    "Id": []
}

In [6]:
# Get the current exercise
current_exercise = df.iloc[0, df.columns.get_loc('activity')]
id = df.iloc[0, df.columns.get_loc('id')]
max_soc, min_soc = 0, 101
power_list = [0]
min_time, max_time = 100, 0

testing_purpose = []

# iterate over all the rows
for index, row in df.iterrows():

    # Get the data needed from the rows. Append the power
    new_exercise = row["activity"]
    soc = row["soc"]
    power_list.append(row["power"])
    time = row["time"]

    if new_exercise not in testing_purpose:
        print(f"We are on this exercise: {new_exercise} ------------")
        testing_purpose.append(new_exercise)

    # If the exercise changes or if the rows end.
    if current_exercise != new_exercise or index == len(df) - 1:

        print(f"Testing exercise: {current_exercise}")

        # Set the values 
        data_dict["Time Delta"].append(round(max_time - min_time, 2))
        data_dict["SOC Delta"].append(max_soc - min_soc)
        data_dict["Activity"].append(current_exercise)
        data_dict["Average Power"].append(round(sum(power_list)/len(power_list), 2))
        data_dict["Id"].append(id)

        # Reset all the values
        max_soc, min_soc = soc, soc
        power_list.clear()
        max_time, min_time = time, time
    
    # SOC
    if soc >= max_soc:
        max_soc = soc
    if soc <= min_soc:
        min_soc = soc

    # TIME
    if time >= max_time:
        max_time = time
    if time <= min_time:
        min_time = time
    
    # Change current exercise
    current_exercise = new_exercise

print(data_dict)


We are on this exercise: takeoff ------------
We are on this exercise: climb ------------
Testing exercise: takeoff
We are on this exercise: cruise ------------
Testing exercise: climb
We are on this exercise: landing ------------
Testing exercise: cruise
Testing exercise: landing
Testing exercise: takeoff
Testing exercise: climb
Testing exercise: cruise
Testing exercise: landing
Testing exercise: takeoff
Testing exercise: climb
Testing exercise: cruise
Testing exercise: landing
Testing exercise: takeoff
Testing exercise: climb
We are on this exercise: descent ------------
Testing exercise: cruise
Testing exercise: descent
Testing exercise: landing
Testing exercise: takeoff
Testing exercise: climb
Testing exercise: cruise
Testing exercise: descent
Testing exercise: landing
Testing exercise: takeoff
Testing exercise: climb
Testing exercise: cruise
Testing exercise: descent
Testing exercise: landing
{'Time Delta': [1.0, 0.9, 1.3, 1.2, 0.7, 1.0, 1.9, 1.4, 0.9, 0.66, 2.4, 1.7, 1.0, 0.6, 1.

In [7]:
final_df = pd.DataFrame(data_dict)
print(final_df)

    Time Delta  SOC Delta Activity  Average Power    Id
0         1.00          3  takeoff          53.39  4620
1         0.90          4    climb          51.92  4620
2         1.30          2   cruise          15.35  4620
3         1.20          0  landing           0.08  4620
4         0.70          3  takeoff          53.31  4620
5         1.00          4    climb          44.63  4620
6         1.90          3   cruise          15.12  4620
7         1.40          0  landing           0.04  4620
8         0.90          3  takeoff          53.05  4620
9         0.66          3    climb          50.81  4620
10        2.40          5   cruise          19.76  4620
11        1.70          0  landing           0.00  4620
12        1.00          4  takeoff          56.48  4620
13        0.60          3    climb          51.49  4620
14        1.50          3   cruise          20.41  4620
15        1.00          0  descent           3.46  4620
16        1.50          0  landing           0.5

In [ ]:
# Flight IDs to include
flight_ids = [4620, 4929, 4940, 5019, 5021, 5034]

# Fetch data for specified flight IDs
data_frames = [db_connect.connect_flight_for_ml_data_prescription(flight_id) for flight_id in flight_ids]

# Concatenate data frames and shuffle the data
all_data = pd.concat(data_frames, axis=0).sample(frac=1, random_state=42)

# Split the data into train and test sets
train_data, test_data = train_test_split(all_data, test_size=0.5, random_state=42)

In [ ]:
print(f"Length train_x = {len(train_data)} \n Length train_y = {len(test_data)}")

One-Hot-Encoding of the Operations columns

In [ ]:
# ONE-HOT ENCODE
# https://stackabuse.com/one-hot-encoding-in-python-with-pandas-and-scikit-learn/
def one_hot(df, col, pre):
  encoded = pd.get_dummies(df[col], prefix=pre)
  for column in encoded:
    encoded = encoded.rename(columns={column: col + "_" + column})
  encoded['time'] = df['time']
  encoded["id"] = df["id"]
  return encoded

In [ ]:
# Encode Train data
train_encoded = one_hot(train_data, "exercise", 'is')
final_train_x = pd.merge(train_data, train_encoded, on=["time", "id"])
final_train_y = final_train_x["soc"].to_numpy()
final_train_x = final_train_x.drop(columns=["time", "id", "exercise", "soc"])

# Encode Test data
test_encoded = one_hot(test_data, "exercise", 'is')
final_test_x = pd.merge(test_data, test_encoded, on=["time","id"])
final_test_y = final_test_x["soc"].to_numpy()
final_test_x = final_test_x.drop(columns=["time", "id", "exercise", "soc"])

In [ ]:
final_train_x

In [ ]:
print(f"Length test_encoded = {len(test_encoded)} \n Length train_encoded = {len(train_encoded)}")

In [ ]:
print(f"Length train_x = {len(final_train_x)} \n Length train_y = {len(final_train_y)}")

In [ ]:
print(f"Length test_x = {len(final_test_x)} \n Length test_y = {len(final_test_y)}")

Machine Learning Model Implementation

In [ ]:
# import sklearn
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, classification_report

# Set model
regression_model = LinearRegression()

# Fit model
regression_model.fit(final_train_x, final_train_y)

In [ ]:
# print model score
print(regression_model.score(final_test_x, final_test_y))

In [ ]:
coeff = pd.DataFrame(list(zip(regression_model.feature_names_in_, regression_model.coef_)), columns = ['Feature', 'Weight'])
coeff.sort_values('Weight')

In [ ]:
# Set index and initial exercise
start_index = 0
current_exercise = df.iloc[0, df.columns.get_loc('activity')]

# iterate over all the rows
for index, row in df.iterrows():

    # Initalize new exercise
    new_exercise = row["activity"]

    # See if the exercise changes
    if current_exercise != new_exercise or index == len(df):

        # Get SOC for the exercise and store the change from max to min.
        soc_range = df.iloc[start_index:index-1, df.columns.get_loc('soc')]
        soc_max, soc_min = np.max(soc_range), np.min(soc_range)
        data_dict["SOC Delta"].append(soc_max - soc_min)

        print(f"SOC Max: {soc_max}, SOC Min: {soc_min}")

        # Get Time for the exercise and store the change from max to min.
        time_range = df.iloc[start_index:index-1, df.columns.get_loc('time')]
        time_start, time_end = np.min(time_range), np.max(time_range)
        data_dict["Time Delta"].append(round(time_end - time_start, 2))

        # Get the Average Power
        power_average = np.mean(df.iloc[start_index:index-1, df.columns.get_loc('power')])
        data_dict["Average Power"].append(round(power_average, 2))

        # Set the id and activity
        data_dict["Id"].append(row["id"])
        data_dict["Activity"].append(current_exercise)

        # Reset the index and exercise to the one 
        start_index = index
        current_exercise = new_exercise
    
    if index == len(df):
        break